### Rubanenko Evgeny, 595

## Check Questions

**Вопрос 1:** Чем LSTM лучше/хуже чем обычная RNN?  
**Ответ:** Основная проблема RNN - затухающий / взрывающийся градиент. LSTM намного более устойчива к этой проблеме - поэтому ее и стали использовать. Но у нее есть тоже свои минусы - долгие вычисления, множество параметров. Так же LSTM может потерять эффективность, если делать много слоев (обычно делают 2-3). Но сделать RNN, которая будет лучше чем LSTM (так называемая Vanilla RNN) очень сложно.

**Вопрос 2:** Выпишите производную $\frac{\partial c_{n+1}}{\partial c_{k}}$ для LSTM http://colah.github.io/posts/2015-08-Understanding-LSTMs/, объясните формулу, когда производная затухает, когда взрывается?  
**Ответ:** Будем вместо $c_{n}$ писать $h_{n}$ - так привычнее.  
Известно следующее:
$$h_{t} = W \cdot f(h_{t-1}) + W^{(hx)} \cdot x_{[t]}$$
Тогда для $\frac{\partial h_{n+1}}{\partial h_{k}}$ по chain rule сможем написать следующее:
$$\frac{\partial h_{n+1}}{\partial h_{k}} = \prod_{l=k}^{n} \frac{\partial h_{l+1}}{\partial h_{l}}$$
По формуле выше сможем переписать произвольную производную из этого произведения следующим образом:
$$\frac{\partial h_{l+1}}{\partial h_{l}} = W^{T} f'(h_{l})$$
Здесь так же необходимо учесть, что последняя производная на самом деле является Якобианом:
$$f'(h_{l}) = \left(\frac{\partial f_{i}}{\partial x_{j}}\right)_{(i,j)}, i \in \{1..m\},\ j \in \{1..n\}$$
Учитывая написанное, мы можем оценить норму частной производной:
$$\left|\left|\frac{\partial h_{l+1}}{\partial h_{l}}\right|\right| \leq \left|\left|W^{T}\right|\right| \cdot \left|\left|diag(f'(h_{l}))\right|\right| \leq \beta_{W}\beta_{h}$$
где $\beta$ есть верхняя граница соответствующей нормы. Тогда для исходной частной производной получаем:
$$\left|\left|\frac{\partial h_{n+1}}{\partial h_{k}}\right|\right| = \left|\left|\prod_{l=k}^{n} \frac{\partial h_{l+1}}{\partial h_{l}}\right|\right| \leq (\beta_{W}\beta_{h})^{n - k + 1}$$
Если $\beta_{W}\beta_{h}$ меньше 1, то при увеличении n будем наблюдать затухающий градиент, а если больше, то взрывающийся.

**Вопрос 3:** Зачем нужен TBPTT? Почему BPTT плох?  
**Ответ:** Основной проблемой BPTT является стоимость обновления одного параметра - она большая. Из-за этого становится невозможным производить много итераций. Другая проблема - если вход большой, то для одного обновления придется посчитать большое число производных - возникает проблема затухающего / взрывающегося градиента. Победить все это был призван TBPTT. TBPTT разбивает длинный вход на несколько маленьких по длине, и работает с новыми входами как с отдельными задачами. Тогда, если разбивать длинный вход на много маленьких - можно избавиться от проблем стоимости. Также станет больше hidden states - дополнительная информация о далеком прошлом может оказаться полезной.

**Вопрос 4:** Как комбинировать рекуррентные и сверточные сети, а главное зачем? Приведите несколько примеров реальных задач.  
**Ответ:** Идея состоит в том, что CNN обучается на каких-то пространственных объектах, после чего получаются какие-то их высокоуровневые описания, которые потом подаются на вход RNN, которая либо подробно описывает их, либо просто классифицирует. Комбинирование используется, потому что CNN хорошо работает с пространственно связанными данными, а RNN - с временными сигналам.

Возможные задачи:
1. По фрагменту видео классифиировать происходящее
2. По изображению объекта получить его текстовое описание

В последнем случае CNN распознает, что есть на изображении, передает эту информацию в RNN, которая уже генерирует текстовое описание.

**Вопрос 5:** Можно ли использовать сверточные сети для классификации текстов? Как решить проблему с произвольной длинной входа?  
**Ответ:** Это не является традиционным подходом, но возможно. Сначала текст пройдет некоторый lookup-table (посчитаются вхождения символов из входного текста), а затем пойдут обычные слои CNN, длина текста считается фиксированной. При несовпадении текст либо заполнится какими-то dummy символами, либо разобьется на вектор текстов.

**Вопрос 6:** Attention - что это такое, где применяют и как? Приведите пример использования на какой-нибудь задаче.  
**Ответ:** Attention применяют в RNN, чтобы побороть типичный bottleneck у RNN: последняя ячейка encoder должна знать всю информацию об input. Идея состоит в том, чтобы давать важным частям входа более высокие веса (т.е. decoder уже не просто смотрит на то, что пришло ему от последнего encoder, а на каждом шаге обращается к нужной части входа).  
Типичное применение - NMT. Применяется в NMT, потому что маловерятно, что последний encoder запомнит информацию о первом слове в предложении (например, если вход имел длину 50, то последний encoder должен помнить прошлые 50 шагов). LSTM должны решать проблему с длительными зависимостями, но делают это недостаточно эффективно. Используя Attention, decoder при генерации первого слова обращает больше внимания на первое слово во входе и тд.  
Используют также при генерации описания к изображению: CNN кодирует изображение, RNN генерирует описание, а Attention позволяет сильнее фокусироваться на том, описание чего сейчас генерируется.

In [3]:
!pip install faker tqdm babel

In [4]:
import random
import numpy as np
import keras.backend as K

from tqdm import tqdm
from babel.dates import format_date
from faker import Faker

from keras.layers import Embedding, Bidirectional
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *
from keras.layers.merge import Multiply
from keras.utils import to_categorical
from keras.layers import TimeDistributed, Multiply

Using TensorFlow backend.


In [0]:
fake = Faker()

FORMATS = ['short',
           'medium',
           'long',
           'full',
           'd MMM YYY', 
           'd MMMM YYY',
           'dd MMM YYY',
           'd MMM, YYY',
           'd MMMM, YYY',
           'dd, MMM YYY',
           'd MM YY',
           'd MMMM YYY',
           'MMMM d YYY',
           'MMMM d, YYY',
           'dd.MM.YY']

LOCALES = ['en_US']

In [0]:
def create_date():
    '''
        Creates some fake dates 
        Returns: tuple containing human readable string,
                 machine readable string, and date object
    '''
    dt = fake.date_object()

    try:
        human_readable = format_date(dt,
                                     format=random.choice(FORMATS),
                                     locale=random.choice(LOCALES))

        case_change = random.choice([0,1,2])
        if case_change == 1:
            human_readable = human_readable.upper()
        elif case_change == 2:
            human_readable = human_readable.lower()

        machine_readable = dt.isoformat()
    except AttributeError as e:
        return None, None, None

    return human_readable, machine_readable, dt

In [0]:
def create_dataset(n_examples):
    '''
        Creates a dataset with n_examples and vocabularies
        :n_examples: the number of examples to generate
    '''
    human_vocab = set()
    machine_vocab = set()
    dataset = []

    for i in tqdm(range(n_examples)):
        h, m, _ = create_date()
        if h is not None:
            dataset.append((h, m))
            human_vocab.update(tuple(h))
            machine_vocab.update(tuple(m))

    human = dict(zip(list(human_vocab) + ['<unk>', '<pad>'], 
                     list(range(len(human_vocab) + 2))))
    inv_machine = dict(enumerate(list(machine_vocab) + ['<unk>', '<pad>']))
    machine = {v:k for k,v in inv_machine.items()}
 
    return dataset, human, machine, inv_machine

In [0]:
def string_to_int(string, lenght, vocab):
    if len(string) > lenght:
        string = string[:lenght]
        
    rep = list(map(lambda x: vocab.get(x, '<unk>'), string))
    
    if len(string) < lenght:
        rep += [vocab['<pad>']] * (lenght - len(string))
    
    return rep

In [0]:
def int_to_string(ints, inv_vocab):
    return [inv_vocab[i] for i in ints]

### Actual data generation

In [11]:
fake.seed(42)
random.seed(42)
N = int(3e5)
dataset, human_vocab, machine_vocab, inv_machine_vocab = create_dataset(N)

 54%|█████▍    | 161348/300000 [00:09<00:08, 16173.80it/s]/usr/local/lib/python3.6/dist-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 300000/300000 [00:18<00:00, 16134.85it/s]


In [12]:
dataset[2]

('tuesday, september 14, 1971', '1971-09-14')

In [0]:
inputs, targets = zip(*dataset)
inputs = np.array([string_to_int(i, TIME_STEPS, human_vocab) for i in inputs])
targets = [string_to_int(t, TIME_STEPS, machine_vocab) for t in targets]
targets = np.array(list(map(lambda x: 
                            to_categorical(x, num_classes=len(machine_vocab)), 
                            targets)))

In [0]:
X_train, y_train, X_valid, y_valid, X_test, y_test = (
    inputs[:int(2e5)], targets[:int(2e5)], 
    inputs[int(2e5):-int(5e4)], targets[int(2e5):-int(5e4)],  
    inputs[-int(5e4):], targets[-int(5e4):])

### Simple NMT

In [0]:
EMBEDING = 128
ENCODER_UNITS = 128
DECODER_UNITS = 128
TIME_STEPS = 20

In [0]:
# Моя первая модель. Дала 86%. Через пару часов я понял,
# что просто две LSTM подряд запустил. Так что так себе достижение.
def model_simple_nmt_1(in_chars, out_chars):
    # RNN encoder -> hidden representation -> RNN decoder
    inputs = Input(shape=(TIME_STEPS,))
    encoder1 = Embedding(in_chars, ENCODER_UNITS)(inputs)
    encoder2 = LSTM(ENCODER_UNITS, return_sequences=True)(encoder1)
    decoder1 = LSTM(DECODER_UNITS, return_sequences=True)(encoder2)
    outputs = TimeDistributed(Dense(out_chars, activation='softmax'))(decoder1)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [0]:
def model_simple_nmt(in_chars, out_chars):
    # RNN encoder -> hidden representation -> RNN decoder
    inputs = Input(shape=(TIME_STEPS,))
    encoder1 = Embedding(in_chars, EMBEDING)(inputs)
    encoder2 = LSTM(ENCODER_UNITS)(encoder1)
    encoder3 = RepeatVector(TIME_STEPS)(encoder2)
    decoder1 = LSTM(DECODER_UNITS, return_sequences=True)(encoder3)
    outputs = TimeDistributed(Dense(out_chars, activation='softmax'))(decoder1)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [16]:
m = model_simple_nmt(len(human_vocab), len(machine_vocab))
m.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])
print(m.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 20, 128)           7680      
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 20, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 13)            1677      
Total params: 272,525
Trainable params: 272,525
Non-trainable params: 0
_________________________________________________________________
None

In [17]:
m.fit(x=[X_train], y=y_train, validation_data=(X_valid, y_valid),
    epochs=10, batch_size=64, validation_split=0.1)

Train on 200000 samples, validate on 50000 samples
Epoch 1/10
 42560/200000 [=====>........................] - ETA: 4:23 - loss: 0.7954 - acc: 0.7149

128896/200000 [==================>...........] - ETA: 1:58 - loss: 0.4238 - acc: 0.8442

200000/200000 [==============================] - 352s 2ms/step - loss: 0.2863 - acc: 0.8957 - val_loss: 0.0219 - val_acc: 0.9923
Epoch 2/10
   640/200000 [..............................] - ETA: 5:30 - loss: 0.0268 - acc: 0.9905

 83520/200000 [===========>..................] - ETA: 3:11 - loss: 0.0202 - acc: 0.9923

161536/200000 [=======================>......] - ETA: 1:03 - loss: 0.0201 - acc: 0.9922

...

156096/200000 [======================>.......] - ETA: 1:10 - loss: 0.0155 - acc: 0.9931

200000/200000 [==============================] - 341s 2ms/step - loss: 0.0155 - acc: 0.9931 - val_loss: 0.0155 - val_acc: 0.9930
Epoch 10/10
 10176/200000 [>.............................] - ETA: 5:06 - loss: 0.0161 - acc: 0.9926

 83392/200000 [===========>..................] - ETA: 3:07 - loss: 0.0155 - acc: 0.9930

154368/200000 [======================>.......] - ETA: 1:13 - loss: 0.0155 - acc: 0.9931

200000/200000 [==============================] - 340s 2ms/step - loss: 0.0154 - acc: 0.9931 - val_loss: 0.0154 - val_acc: 0.9930


In [18]:
m.evaluate([X_test], y_test)

50000/50000 [==============================] - 45s 891us/step


[0.015360312386206642, 0.993050000076294]

In [0]:
EXAMPLES = ['3 May 1979', '5 Apr 09', '20th February 2016', 'Wed 10 Jul 2007']

def run_example(model, input_vocabulary, inv_output_vocabulary, text):
    encoded = string_to_int(text, TIME_STEPS, input_vocabulary)
    prediction = model.predict(np.array([encoded]))
    prediction = np.argmax(prediction[0], axis=-1)
    return int_to_string(prediction, inv_output_vocabulary)

def run_examples(model, input_vocabulary,
                 inv_output_vocabulary, examples=EXAMPLES):
    predicted = []
    for example in examples:
        predicted.append(''.join(run_example(model, input_vocabulary,
                                             inv_output_vocabulary, example)))
        print('input:', example)
        print('output:', predicted[-1])
    return predicted

In [20]:
run_examples(m, human_vocab, inv_machine_vocab)

input: 3 May 1979
output: 1979-05-03<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
input: 5 Apr 09
output: 2009-04-05<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
input: 20th February 2016
output: 2016-02-20<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
input: Wed 10 Jul 2007
output: 2007-07-10<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


['1979-05-03<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '2009-04-05<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '2016-02-20<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '2007-07-10<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>']

### NMT with attention

In [0]:
EMBEDING = 128
ENCODER_UNITS = 256
DECODER_UNITS = 256
TIME_STEPS = 20

In [0]:
def attention_cell(inputs):
    permutation = Permute((2, 1))(inputs)
    dense = Dense(TIME_STEPS, activation='softmax')(permutation)
    probs = Permute((2, 1))(dense)
    output_attention_mul = Multiply()([inputs, probs])
    return output_attention_mul

In [0]:
def model_attention_nmt(in_chars, out_chars):
    # RNN encoder -> hidden representation -> RNN decoder
    inputs = Input(shape=(TIME_STEPS,))
    embeding = Embedding(in_chars, EMBEDING)(inputs)
    lstm = LSTM(ENCODER_UNITS)(embeding)
    repeat = RepeatVector(TIME_STEPS)(lstm)
    lstm = LSTM(DECODER_UNITS, return_sequences=True)(repeat)
    attention = attention_cell(lstm)
    outputs = TimeDistributed(Dense(out_chars, activation='softmax'))(attention)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [38]:
m = model_attention_nmt(len(human_vocab), len(machine_vocab))
m.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])
print(m.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 20, 128)      7680        input_7[0][0]                    
__________________________________________________________________________________________________
lstm_11 (LSTM)                  (None, 256)          394240      embedding_6[0][0]                
__________________________________________________________________________________________________
repeat_vector_6 (RepeatVector)  (None, 20, 256)      0           lstm_11[0][0]                    
__________________________________________________________________________________________________
lstm_12 (L

In [39]:
m.fit([X_train], y_train, validation_data=(X_valid, y_valid),
    epochs=10, batch_size=64, validation_split=0.1)

Train on 200000 samples, validate on 50000 samples
Epoch 1/10
 42880/200000 [=====>........................] - ETA: 4:26 - loss: 0.9509 - acc: 0.6936

129600/200000 [==================>...........] - ETA: 1:57 - loss: 0.7253 - acc: 0.7283

200000/200000 [==============================] - 351s 2ms/step - loss: 0.6789 - acc: 0.7344 - val_loss: 0.5894 - val_acc: 0.7456
Epoch 2/10
  1024/200000 [..............................] - ETA: 5:22 - loss: 0.5914 - acc: 0.7448

 73792/200000 [==========>...................] - ETA: 3:26 - loss: 0.5057 - acc: 0.7968

152448/200000 [=====================>........] - ETA: 1:17 - loss: 0.4423 - acc: 0.8263

...

 85888/200000 [===========>..................] - ETA: 3:02 - loss: 0.0160 - acc: 0.9929

157504/200000 [======================>.......] - ETA: 1:08 - loss: 0.0157 - acc: 0.9930

200000/200000 [==============================] - 341s 2ms/step - loss: 0.0157 - acc: 0.9930 - val_loss: 0.0154 - val_acc: 0.9930
Epoch 10/10
 11072/200000 [>.............................] - ETA: 5:08 - loss: 0.0157 - acc: 0.9929

146496/200000 [====================>.........] - ETA: 1:28 - loss: 0.0155 - acc: 0.9930

200000/200000 [==============================] - 349s 2ms/step - loss: 0.0155 - acc: 0.9930 - val_loss: 0.0154 - val_acc: 0.9931


In [40]:
m.evaluate([X_test], y_test)

50000/50000 [==============================] - 44s 884us/step


[0.015390844363240758, 0.9930530004501342]

## Report

Simple NMT:
* Какого-то подбора параметров или манипуляций со слоями мне делать не пришлось. Как только я написал правильную модель (со второго раза, первая была неправильной), я сразу получил требуемое accuracy.

Attention NMT:
* Реализовал Attention так, как предлагалось в домашней работе - добавил дополнительный слой. Ничего подбирать не пришлось - сразу получилось нужное accuracy.

Comparison:
* Так как с помощью Simple NMT можно добиться того же результата, что и с помощью Attention NMT, но притом потратить меньше времени, то в данном случае разумнее использовать Simple NMT. Возможно, есть ситуации когда bottleneck от Simple NMT все испортит, но здесь этого не наблюдается.

## Tatoeba

In [11]:
! wget http://www.manythings.org/anki/rus-eng.zip

--2018-05-05 14:17:39--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 104.24.109.196, 2400:cb00:2048:1::6818:6dc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6366669 (6.1M) [application/zip]
Saving to: ‘rus-eng.zip.1’

rus-eng.zip.1       100%[===================>]   6.07M  21.7MB/s    in 0.3s    

2018-05-05 14:17:40 (21.7 MB/s) - ‘rus-eng.zip.1’ saved [6366669/6366669]



In [12]:
! unzip ./rus-eng.zip

Archive:  ./rus-eng.zip
replace rus.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [0]:
with open("./rus.txt") as fin:
    data = fin.readlines()
data = list(map(lambda x: x.replace("\n", "").lower(), data))

In [0]:
data = data[:int(1e5)]

In [0]:
source = list(map(lambda x: x.split("\t")[0], data))
target = list(map(lambda x: x.split("\t")[1], data))

In [0]:
source_vocab = set("".join(source).strip())
target_vocab = set("".join(target).strip())

In [0]:
source_vocab = dict(zip(
    list(source_vocab) + ['<unk>', '<pad>'], 
    list(range(len(source_vocab) + 2))))
target_vocab = dict(zip(
    list(target_vocab) + ['<unk>', '<pad>'], 
    list(range(len(target_vocab) + 2))))
inv_target_vocab = dict(enumerate(list(target_vocab) + ['<unk>', '<pad>']))

In [0]:
def model_simple_nmt_tatoeba(in_chars, out_chars):
    # RNN encoder -> hidden representation -> RNN decoder
    inputs = Input(shape=(TIME_STEPS,))
    encoder1 = Embedding(in_chars, 128)(inputs)
    encoder2 = LSTM(256)(encoder1)
    encoder3 = RepeatVector(TIME_STEPS)(encoder2)
    decoder1 = LSTM(256, return_sequences=True)(encoder3)
    outputs = TimeDistributed(Dense(out_chars, activation='softmax'))(decoder1)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [87]:
m = model_simple_nmt_tatoeba(len(source_vocab), len(target_vocab))

m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(m.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        (None, 20)                0         
_________________________________________________________________
embedding_27 (Embedding)     (None, 20, 128)           6912      
_________________________________________________________________
lstm_48 (LSTM)               (None, 256)               394240    
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 20, 256)           0         
_________________________________________________________________
lstm_49 (LSTM)               (None, 20, 256)           525312    
_________________________________________________________________
time_distributed_9 (TimeDist (None, 20, 87)            22359     
Total params: 948,823
Trainable params: 948,823
Non-trainable params: 0
_________________________________________________________________
None

In [0]:
inputs = np.array([string_to_int(i, TIME_STEPS, source_vocab) for i in source])
targets = [string_to_int(t, TIME_STEPS, target_vocab) for t in target]
targets = np.array(list(map(lambda x: to_categorical(x, num_classes=len(target_vocab)), targets)))

In [89]:
m.fit([inputs], targets, epochs=10, batch_size=64, validation_split=0.1)

Train on 90000 samples, validate on 10000 samples
Epoch 1/10
43776/90000 [=============>................] - ETA: 1:19 - loss: 2.8866 - acc: 0.2342

90000/90000 [==============================] - 158s 2ms/step - loss: 2.7862 - acc: 0.2602 - val_loss: 2.8548 - val_acc: 0.2285
Epoch 2/10
11200/90000 [==>...........................] - ETA: 2:14 - loss: 2.6033 - acc: 0.3069

89984/90000 [============================>.] - ETA: 0s - loss: 2.5313 - acc: 0.3237

90000/90000 [==============================] - 159s 2ms/step - loss: 2.5313 - acc: 0.3237 - val_loss: 2.6690 - val_acc: 0.2743
Epoch 3/10
28608/90000 [========>.....................] - ETA: 1:44 - loss: 2.4329 - acc: 0.3480

90000/90000 [==============================] - 159s 2ms/step - loss: 2.3937 - acc: 0.3572 - val_loss: 2.5728 - val_acc: 0.2981
Epoch 4/10
 5376/90000 [>.............................] - ETA: 2:24 - loss: 2.3294 - acc: 0.3731

...

89984/90000 [============================>.] - ETA: 0s - loss: 2.0328 - acc: 0.4357

90000/90000 [==============================] - 160s 2ms/step - loss: 2.0328 - acc: 0.4357 - val_loss: 2.4027 - val_acc: 0.3420
Epoch 9/10
28608/90000 [========>.....................] - ETA: 1:44 - loss: 1.9882 - acc: 0.4442

90000/90000 [==============================] - 158s 2ms/step - loss: 1.9870 - acc: 0.4453 - val_loss: 2.4148 - val_acc: 0.3417
Epoch 10/10
 5376/90000 [>.............................] - ETA: 2:23 - loss: 1.9339 - acc: 0.4584

89984/90000 [============================>.] - ETA: 0s - loss: 1.9465 - acc: 0.4540

90000/90000 [==============================] - 158s 2ms/step - loss: 1.9465 - acc: 0.4540 - val_loss: 2.4017 - val_acc: 0.3444


In [91]:
run_example(m, source_vocab, inv_target_vocab, 'hello')

['п',
 'о',
 'о',
 'о',
 'о',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']

In [0]:
def model_simple_nmt_tatoeba(in_chars, out_chars):
    # RNN encoder -> hidden representation -> RNN decoder
    inputs = Input(shape=(TIME_STEPS,))
    encoder1 = Embedding(in_chars, 128)(inputs)
    encoder2 = LSTM(300, return_sequences=True)(encoder1)
    encoder3 = LSTM(300, return_sequences=False)(encoder2)
    encoder4 = RepeatVector(TIME_STEPS)(encoder3)
    decoder1 = LSTM(300, return_sequences=True)(encoder4)
    outputs = TimeDistributed(Dense(out_chars, activation='softmax'))(decoder1)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [28]:
m = model_simple_nmt_tatoeba(len(source_vocab), len(target_vocab))

m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(m.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 20, 128)           6912      
_________________________________________________________________
lstm_4 (LSTM)                (None, 20, 300)           514800    
_________________________________________________________________
lstm_5 (LSTM)                (None, 300)               721200    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 20, 300)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 20, 300)           721200    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 87)            26187     
Total para

In [29]:
m.fit([inputs], targets, epochs=10, batch_size=64, validation_split=0.1)

Train on 90000 samples, validate on 10000 samples
Epoch 1/10
43776/90000 [=============>................] - ETA: 1:52 - loss: 2.8896 - acc: 0.2344

90000/90000 [==============================] - 223s 2ms/step - loss: 2.7936 - acc: 0.2593 - val_loss: 2.8449 - val_acc: 0.2291
Epoch 2/10
11136/90000 [==>...........................] - ETA: 3:08 - loss: 2.6189 - acc: 0.3043

89984/90000 [============================>.] - ETA: 0s - loss: 2.5450 - acc: 0.3206

90000/90000 [==============================] - 222s 2ms/step - loss: 2.5450 - acc: 0.3206 - val_loss: 2.6828 - val_acc: 0.2718
Epoch 3/10
28608/90000 [========>.....................] - ETA: 2:26 - loss: 2.4379 - acc: 0.3465

90000/90000 [==============================] - 221s 2ms/step - loss: 2.4022 - acc: 0.3555 - val_loss: 2.5743 - val_acc: 0.2997
Epoch 4/10
 5312/90000 [>.............................] - ETA: 3:23 - loss: 2.3327 - acc: 0.3703

...

89984/90000 [============================>.] - ETA: 0s - loss: 1.9797 - acc: 0.4463

90000/90000 [==============================] - 220s 2ms/step - loss: 1.9797 - acc: 0.4463 - val_loss: 2.3745 - val_acc: 0.3482
Epoch 9/10
28608/90000 [========>.....................] - ETA: 2:29 - loss: 1.9217 - acc: 0.4580

90000/90000 [==============================] - 222s 2ms/step - loss: 1.9207 - acc: 0.4585 - val_loss: 2.3688 - val_acc: 0.3505
Epoch 10/10
 5312/90000 [>.............................] - ETA: 3:19 - loss: 1.8526 - acc: 0.4725

89984/90000 [============================>.] - ETA: 0s - loss: 1.8656 - acc: 0.4705

90000/90000 [==============================] - 220s 2ms/step - loss: 1.8656 - acc: 0.4705 - val_loss: 2.3698 - val_acc: 0.3544


In [32]:
run_example(m, source_vocab, inv_target_vocab, 'hello')

['п',
 'р',
 'п',
 'о',
 'а',
 'т',
 'т',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']

In [0]:
TIME_STEPS = 50
EMBEDING = 256
ENCODER_UNITS = 512
DECODER_UNITS = 512

In [0]:
inputs = np.array([string_to_int(i, TIME_STEPS, source_vocab) for i in source])
targets = [string_to_int(t, TIME_STEPS, target_vocab) for t in target]
targets = np.array(list(map(lambda x: to_categorical(x, num_classes=len(target_vocab)), targets)))

In [0]:
def model_attention_nmt_tatoeba(in_chars, out_chars):
    # RNN encoder -> hidden representation -> RNN decoder
    inputs = Input(shape=(TIME_STEPS,))
    embeding = Embedding(in_chars, EMBEDING)(inputs)
    lstm = LSTM(ENCODER_UNITS, return_sequences=True)(embeding)
    attention = attention_cell(lstm)
    lstm = LSTM(ENCODER_UNITS, return_sequences=True)(attention)
    lstm = LSTM(DECODER_UNITS, return_sequences=True)(lstm)
    outputs = TimeDistributed(Dense(out_chars, activation='softmax'))(lstm)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [23]:
m = model_attention_nmt_tatoeba(len(source_vocab), len(target_vocab))

m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(m.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 50, 256)      13824       input_2[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 50, 512)      1574912     embedding_2[0][0]                
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 512, 50)      0           lstm_2[0][0]                     
__________________________________________________________________________________________________
dense_1 (D

In [24]:
m.fit([inputs], targets, epochs=10, batch_size=256, validation_split=0.1)

Train on 90000 samples, validate on 10000 samples
Epoch 1/10
90000/90000 [==============================] - 230s 3ms/step - loss: 1.4614 - acc: 0.6491 - val_loss: 1.6260 - val_acc: 0.5873
Epoch 2/10
43520/90000 [=============>................] - ETA: 1:53 - loss: 1.3088 - acc: 0.6661

90000/90000 [==============================] - 227s 3ms/step - loss: 1.2923 - acc: 0.6694 - val_loss: 1.5623 - val_acc: 0.5998
Epoch 3/10
90000/90000 [==============================] - 227s 3ms/step - loss: 1.2335 - acc: 0.6811 - val_loss: 1.5126 - val_acc: 0.6093
Epoch 4/10
 3840/90000 [>.............................] - ETA: 3:29 - loss: 1.2057 - acc: 0.6866

90000/90000 [==============================] - 227s 3ms/step - loss: 1.1834 - acc: 0.6927 - val_loss: 1.4817 - val_acc: 0.6183
Epoch 5/10
81664/90000 [==========================>...] - ETA: 20s - loss: 1.1371 - acc: 0.7040

90000/90000 [==============================] - 227s 3ms/step - loss: 1.1349 - acc: 0.7045 - val_loss: 1.4173 - val_acc: 0.6325
Epoch 6/10
90000/90000 [==============================] - 227s 3ms/step - loss: 1.0905 - acc: 0.7140 - val_loss: 1.3907 - val_acc: 0.6376
Epoch 7/10
12800/90000 [===>..........................] - ETA: 3:08 - loss: 1.0628 - acc: 0.7197

90000/90000 [==============================] - 227s 3ms/step - loss: 1.0537 - acc: 0.7210 - val_loss: 1.3725 - val_acc: 0.6428
Epoch 8/10
84992/90000 [===========================>..] - ETA: 12s - loss: 1.0203 - acc: 0.7271

90000/90000 [==============================] - 227s 3ms/step - loss: 1.0196 - acc: 0.7273 - val_loss: 1.3464 - val_acc: 0.6480
Epoch 9/10
90000/90000 [==============================] - 227s 3ms/step - loss: 0.9873 - acc: 0.7328 - val_loss: 1.3330 - val_acc: 0.6503
Epoch 10/10
13568/90000 [===>..........................] - ETA: 3:06 - loss: 0.9596 - acc: 0.7376

90000/90000 [==============================] - 227s 3ms/step - loss: 0.9564 - acc: 0.7386 - val_loss: 1.3222 - val_acc: 0.6527


In [25]:
run_example(m, source_vocab, inv_target_vocab, 'hello')

['п',
 'о',
 'с',
 'о',
 'е',
 'и',
 'е',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']

## Report

1. Решил попробовать запустить здесь Simple NMT. Увеличил число нейронов в сети. Получил на валидации 0.34.
2. Решил добавить слоев в енкодер и декодер. Стало 0.35.
3. Поставил после первой LSTM Attention cell, а потом две LSTM + увеличил TIME_STEPS + увеличил batch_size (потому что долго считалось). Получил 0.65 на валидации. Перевод правда так себе.

*Еще я пробовал использовать Bidirectional, комбинации нескольких LSTM, "подавать все output на вход следующему слою, или только один", но получал что-то в районе 0.44.*